In [31]:
import astropy.units as u
from astroquery.xmatch import XMatch
from astropy.table import Table
from astroquery.vizier import Vizier
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
import matplotlib.pyplot as plt
from astroquery.xmatch import XMatch
import numpy as np
import pandas as pd
from astropy.io import ascii

coord = SkyCoord("04:30:47.99", "+64:50:56.5", unit=(u.hourangle, u.deg), frame='icrs')
rad = 6 * u.arcminute
"""
width = u.Quantity(0.03, u.hourangle)
height = u.Quantity(0.03, u.deg)
Gaia.ROW_LIMIT = -1
r = Gaia.query_object_async(coordinate=coord, width=width, height=height)

'RA_ICRS', 'DE_ICRS', 'Gmag', 'Jmag', 'Hmag', 'Kmag', 'W1mag', 'W2mag', 'W3mag', 'W4mag', 'gmag', 'rmag'
catalogs = ['Gaia', '2MASS', 'MSX', 'WISE', 'PanStarrs']
mag = np.array([['Gmag'],
                ['Jmag', 'Hmag', 'Kmag'],
                ['W1mag', 'W2mag', 'W3mag', 'W4mag'],
                ['gmag', 'rmag']])
"""
catalogs = ['vizier:I/350/gaiaedr3', 'vizier:II/246/out', 'vizier:II/328/allwise', 'vizier:II/349/ps1']
catalog = ['I/350/gaiaedr3', 'II/246/out', 'II/328/allwise', 'II/349/ps1']

df_gaia = Table(names=('RA_ICRS','DE_ICRS'))

data_2mass = Table()
data_wise = Table()
data_panstarrs = Table()
    #names=('RA_ICRS', 'DE_ICRS', 'Gmag', 'Jmag', 'Hmag', 'Kmag', 'W1mag', 'W2mag', 'W3mag', 'W4mag', 'gmag', 'rmag'))

df_gaia = Table(names=('RA_ICRS', 'DE_ICRS', 'Gmag'))
df_2mass = Table(names=('RAJ2000', 'DEJ2000', 'Jmag', 'Hmag', 'Kmag'))
df_wise = Table(names=('RAJ2000', 'DEJ2000', 'W1mag', 'W2mag', 'W3mag', 'W4mag'))
df_panstarrs = Table(names=('RAJ2000', 'DEJ2000', 'gmag', 'rmag'))

for i in range(len(catalogs)):
    result_table = Vizier.query_region(coord, radius=rad, catalog=catalog[i])
    result_table.pprint(show_unit=True, max_lines=100, max_width=1000)
    if i==0:
        df_gaia = result_table[0]['RA_ICRS','DE_ICRS', 'Gmag']
    if i==1:
        print(result_table[0].columns)
        df_2mass = result_table[0]['RAJ2000', 'DEJ2000', 'Jmag', 'Hmag', 'Kmag']
    if i==2:
        df_wise = result_table[0]['RAJ2000', 'DEJ2000', 'W1mag', 'W2mag', 'W3mag', 'W4mag']
    if i==3:
        df_panstarrs = result_table[0]['RAJ2000', 'DEJ2000', 'gmag', 'rmag']
        
table = XMatch.query(cat1=df_gaia,
                     cat2=catalogs[1],
                     max_distance=5 * u.arcsec,
                     colRA1='RA_ICRS', colDec1='DE_ICRS')
data_2mass.add_columns([table ['RA_ICRS'], table['DE_ICRS'], table['Gmag'], table['Jmag'], table['Hmag']])

table = XMatch.query(cat1=data,
                     cat2=catalogs[2],
                     max_distance=5 * u.arcsec,
                     colRA1='RA_ICRS', colDec1='DE_ICRS')
data_wise.add_columns([table['RA_ICRS'], table['DE_ICRS'], table['Gmag'], table['W1mag'], table['W2mag'], table['W3mag'], table['W4mag']])
ascii.write(data_wise, 'gaia_wise.csv', format='csv', fast_writer=False, overwrite=True)

table = XMatch.query(cat1=data,
                     cat2=catalogs[3],
                     max_distance=5 * u.arcsec,
                     colRA1='RA_ICRS', colDec1='DE_ICRS')
data_panstarrs.add_columns([table['RA_ICRS'], table['DE_ICRS'], table['Gmag'], table['gmag'], table['rmag']])


table_wise = Table.read('gaia_wise.csv')
table = XMatch.query(cat1=data_2mass,
                     cat2=table_wise,
                     max_distance=10*u.arcsec,
                     colRA1='RA_ICRS', colDec1='DE_ICRS',
                     colRA2='RA_ICRS', colDec2='DE_ICRS')
print(table)
"""
for i in range(len(table)):
    plt.scatter(table["RA_ICRS"], r["DE_ICRS"], 
            c=r["phot_g_mean_mag"], s=12)
r['pmdec'] = r['pmdec'].apply(correct_pm)
plt.quiver(r["ra"].filled(0), r["dec"].filled(0), r["pmra"].filled(0), r["pmdec"].filled(0), color='0.4')
plt.xlabel("RA, °")
plt.ylabel("DEC, °")
plt.show()
"""

TableList with 1 tables:
	'0:I/350/gaiaedr3' with 39 column(s) and 50 row(s) 
TableList with 1 tables:
	'0:II/246/out' with 15 column(s) and 50 row(s) 
<TableColumns names=('RAJ2000','DEJ2000','_2MASS','Jmag','e_Jmag','Hmag','e_Hmag','Kmag','e_Kmag','Qflg','Rflg','Bflg','Cflg','Xflg','Aflg')>
TableList with 1 tables:
	'0:II/328/allwise' with 28 column(s) and 50 row(s) 
TableList with 1 tables:
	'0:II/349/ps1' with 35 column(s) and 50 row(s) 


HTTPError: 400 Client Error: Mauvaise Requête for url: http://cdsxmatch.u-strasbg.fr/xmatch/api/v1/sync

In [28]:
table_wise

RA_ICRS,DE_ICRS,Gmag,W1mag,W2mag,W3mag,W4mag
float64,float64,float64,float64,float64,float64,float64
67.65393185732,64.76572693604,18.624233,14.766,14.742,12.083,9.122
67.63863570041,64.76042696261,16.605988,13.981,14.063,12.321,8.824
67.5994214376,64.76052089568,19.531567,15.193,15.218,11.972,8.868
67.62341151427,64.76220729184,19.120481,14.339,14.273,12.798,9.073
67.63421636755,64.77217830975,18.319347,13.67,13.753,12.497,9.067
67.63282146088,64.77188148542,16.968378,13.67,13.753,12.497,9.067
...,...,...,...,...,...,...
67.59771206665,64.79745183534,17.217918,14.629,14.567,12.714,8.971
67.55423971834,64.79207868898,19.499905,15.739,15.508,12.774,8.991
